In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import json
import os, sys
import numpy as np
import pandas as pd

current_path = os.getcwd()
sys.path.append(os.path.join(current_path, '..'))

from adapt_model import parse_experiment_name
from utils.globals import *


In [18]:
!ls $RESULTS_DIR

DAMA  original	results_known_disable_mlp_10.jsonl  ROME


In [14]:
model_name = 'llama_7B'

default_params = {
    'iterative_update': False,
    'mixed_update': True,
    'task': 'gen',
    'post_linear': True,
    'batch_size': 1,
    'orthogonal_constraint': 10.0,
    'no_colinear_vs': True,
    'use_neutral': True
}

variable_parameters = {
    'num_layers': [7, 9, 11],
    'null_dim': [64, 128, 256, 512, 1024],
    'vs_at_last': [True, False]
}



In [39]:
def load_data_file(model_name, method, experiment_name, test_file):
    if experiment_name and 'method' != 'original':
        result_dir = f"{model_name}{experiment_name}"
    else:
        result_dir = model_name
    result_file = os.path.join(RESULTS_DIR, method, result_dir, test_file)
    try:
        with open(result_file, 'r') as f:
            results = json.load(f)
    except FileNotFoundError:
        print(f"File {result_file} not found")
        results = None

    return results

Process generation results

In [25]:
def parse_generation_results(layers, dimensions, results):
    out_row = {'layers': layers, 'dimensions': dimensions}
    for key in ('slope_s', 'intercept_s', 'slope_f', 'intercept_f', 'prob_he', 'prob_she', 'prob_they'):
        if results is None:
            out_row[key] = 0.
        else:
            out_row[key] = results[key]
    return out_row

In [26]:
gen_res = pd.DataFrame(columns=['layers', 'dimensions', 'slope_s', 'intercept_s', 'slope_f', 'intercept_f', 'prob_he', 'prob_she', 'prob_they'])
test_file = 'res_gen_test_dama.json'
# row = {'layers': None,'dimensions': None, 'slope_s': 0. ,'intercept_s': 0., 'slope_f': 0., 'intercept_f': 0.,
#        'prob_he': 0., 'prob_she': 0., 'prob_they': 0.}

tmp_res = load_data_file(model_name, 'original', None, test_file)

gen_res.loc[len(gen_res)] = parse_generation_results(0, 0, tmp_res)

for num_layers in variable_parameters['num_layers']:
    for null_dim in variable_parameters['null_dim']:
        vs_at_last = num_layers != 11
        experiment_name = parse_experiment_name(**default_params, num_layers=num_layers, null_dim=null_dim, vs_at_last=vs_at_last)
        tmp_res = load_data_file(model_name, 'DAMA', experiment_name, test_file)
        gen_res.loc[len(gen_res)] = parse_generation_results(num_layers, null_dim, tmp_res)

In [30]:
display(gen_res)
# save to csv
gen_res.to_csv(os.path.join(RESULTS_DIR, 'gen_res.csv'), index=False)

,layers,dimensions,slope_s,intercept_s,slope_f,intercept_f,prob_he,prob_she,prob_they
0,0,0,0.204384,0.130726,0.770820,0.169416,0.266737,0.072094,0.024262
1,7,64,0.182529,0.096371,0.693667,0.130751,0.222804,0.069352,0.025365
2,7,128,0.183076,0.102495,0.692600,0.137081,0.230931,0.071184,0.025327
3,7,256,0.183076,0.102495,0.692600,0.137081,0.230931,0.071184,0.025327
4,7,512,0.173165,0.097712,0.664149,0.130130,0.226270,0.074404,0.027037
5,7,1024,0.157779,0.093135,0.604006,0.122709,0.214962,0.072486,0.027603
6,9,64,0.167587,0.082449,0.632262,0.114166,0.200563,0.065706,0.026166
7,9,128,0.167858,0.090851,0.631464,0.122678,0.210972,0.067627,0.025233
8,9,256,0.167858,0.090851,0.631464,0.122678,0.210972,0.067627,0.025233
9,9,512,0.150585,0.077217,0.578658,0.105371,0.196754,0.072445,0.029707


Process causal lm results

In [33]:
def parse_causal_lm_results(layers, dimensions, results):
    out_row = {'layers': layers, 'dimensions': dimensions, 'perplexity': results['mean_perplexity'] if results else 0.}
    return out_row

In [35]:
lm_res = pd.DataFrame(columns=['layers', 'dimensions', 'perplexity'])

test_file = 'res_causal_lm_wikitext_wikitext-103-raw-v1.json'
# row = {'layers': None,'dimensions': None, 'perplexity': 0.}

tmp_res = load_data_file(model_name, 'original', None, test_file)
lm_res.loc[len(lm_res)] = parse_causal_lm_results(0, 0, tmp_res)

for num_layers in variable_parameters['num_layers']:
    for null_dim in variable_parameters['null_dim']:
        vs_at_last = num_layers != 11
        experiment_name = parse_experiment_name(**default_params, num_layers=num_layers, null_dim=null_dim, vs_at_last=vs_at_last)
        tmp_res = load_data_file(model_name, 'DAMA', experiment_name, test_file)
        lm_res.loc[len(lm_res)] = parse_causal_lm_results(num_layers, null_dim, tmp_res)

In [36]:
display(lm_res)
# save to csv
lm_res.to_csv(os.path.join(RESULTS_DIR, 'lm_res.csv'), index=False)

,layers,dimensions,perplexity
0,0,0,26.079183
1,7,64,26.938672
2,7,128,27.479856
3,7,256,27.479856
4,7,512,30.803725
5,7,1024,34.600923
6,9,64,25.764616
7,9,128,25.989997
8,9,256,25.989997
9,9,512,30.259833


Process coreference results

In [50]:
def parse_causal_lm_results(layers, dimensions, results_a1, results_a2, results_p1, results_p2):
    out_row = {'layers': layers, 'dimensions': dimensions}

    if results_a1 is None or results_a2 is None or results_p1 is None or results_p2 is None:
        out_row['anti_male'] = 0.
        out_row['anti_female'] = 0.
        out_row['pro_male'] = 0.
        out_row['pro_female'] = 0.
    else:
        out_row['anti_male'] = (results_a1['m_acc'] + results_a2['m_acc']) / 2.
        out_row['anti_female'] = (results_a1['f_acc'] + results_a2['f_acc']) / 2.
        out_row['pro_male'] = (results_p1['m_acc'] + results_p2['m_acc']) / 2.
        out_row['pro_female'] = (results_p1['f_acc'] + results_p2['f_acc']) / 2.

    out_row['ACC'] = (out_row['anti_male'] + out_row['anti_female'] + out_row['pro_male'] + out_row['pro_female']) / 4.
    out_row['Delta_S'] = (out_row['pro_male'] + out_row['pro_female'] - out_row['anti_male'] - out_row['anti_female']) / 2.
    out_row['Delta_G'] = (out_row['pro_male'] + out_row['anti_male'] - out_row['pro_female'] - out_row['anti_female']) / 2.
    return out_row

In [51]:
lm_rs = pd.DataFrame(columns=['layers', 'dimensions', 'ACC', 'Delta_S', 'Delta_G', 'anti_male', 'anti_female', 'pro_male', 'pro_female'])

test_file_a1 = 'res_coref_anti_type1_test.json'
test_file_a2 = 'res_coref_anti_type2_test.json'
test_file_p1 = 'res_coref_pro_type1_test.json'
test_file_p2 = 'res_coref_pro_type2_test.json'


tmp_res_a1 = load_data_file(model_name, 'original', None, test_file_a1)
tmp_res_a2 = load_data_file(model_name, 'original', None, test_file_a2)
tmp_res_p1 = load_data_file(model_name, 'original', None, test_file_p1)
tmp_res_p2 = load_data_file(model_name, 'original', None, test_file_p2)

lm_rs.loc[len(lm_rs)] = parse_causal_lm_results(0, 0, tmp_res_a1, tmp_res_a2, tmp_res_p1, tmp_res_p2)

for num_layers in variable_parameters['num_layers']:
    for null_dim in variable_parameters['null_dim']:
        vs_at_last = num_layers != 11
        experiment_name = parse_experiment_name(**default_params, num_layers=num_layers, null_dim=null_dim, vs_at_last=vs_at_last)
        tmp_res_a1 = load_data_file(model_name, 'DAMA', experiment_name, test_file_a1)
        tmp_res_a2 = load_data_file(model_name, 'DAMA', experiment_name, test_file_a2)
        tmp_res_p1 = load_data_file(model_name, 'DAMA', experiment_name, test_file_p1)
        tmp_res_p2 = load_data_file(model_name, 'DAMA', experiment_name, test_file_p2)
        lm_rs.loc[len(lm_rs)] = parse_causal_lm_results(num_layers, null_dim, tmp_res_a1, tmp_res_a2, tmp_res_p1, tmp_res_p2)



In [52]:
display(lm_rs)
# save to csv
lm_rs.to_csv(os.path.join(RESULTS_DIR, 'coref_res.csv'), index=False)

,layers,dimensions,ACC,Delta_S,Delta_G,anti_male,anti_female,pro_male,pro_female
0,0,0,0.591334,0.403190,0.029478,0.404066,0.375412,0.808081,0.777778
1,7,64,0.589453,0.412004,0.018165,0.391515,0.375387,0.805556,0.785354
2,7,128,0.588178,0.404452,0.015538,0.398965,0.372939,0.792929,0.787879
3,7,256,0.588178,0.404452,0.015538,0.398965,0.372939,0.792929,0.787879
4,7,512,0.582566,0.387898,0.011711,0.421793,0.355442,0.755051,0.797980
5,7,1024,0.569883,0.317305,0.011699,0.462071,0.360390,0.689394,0.767677
6,9,64,0.595772,0.424617,0.028253,0.401616,0.365311,0.818182,0.797980
7,9,128,0.591385,0.415714,0.014275,0.391490,0.375567,0.805556,0.792929
8,9,256,0.591385,0.415714,0.014275,0.391490,0.375567,0.805556,0.792929
9,9,512,0.592668,0.360118,0.024184,0.431742,0.393476,0.777778,0.767677
